<a href="https://colab.research.google.com/github/AdeelAliYousaf/ViolenceDetectionSystem/blob/main/ViolenceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from skimage.transform import resize
from keras.models import load_model
import os
import time
import matplotlib.pyplot as plt
import gradio as gr

model_path = "ViolenceDetectionModel.keras"
model = load_model(model_path, compile=False)
print("[INFO] Pre-trained model loaded successfully.")

def load_video_frames(path, max_frames=30, target_size=(160, 160)):
    cap = cv2.VideoCapture(path)
    frames = []

    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = resize(frame, target_size, preserve_range=True).astype(np.float32) / 255.0
        frames.append(frame)

    cap.release()

    while len(frames) < max_frames:
        frames.append(np.zeros((target_size[0], target_size[1], 3), dtype=np.float32))

    return np.array(frames)

def predict_violence(video_path, threshold=0.65):
    frames = load_video_frames(video_path)
    frames = np.expand_dims(frames, axis=0)
    start = time.time()
    preds = model.predict(frames)[0]
    end = time.time()

    print(f"[INFO] Prediction took {end - start:.2f}s")
    print(f"[INFO] Predicted probabilities: {preds}")

    is_violent = preds[1] >= threshold
    print(f"[RESULT] Violence detected: {is_violent} (Confidence: {preds[1]:.4f})")

    return is_violent, preds[1]

def gradio_predict(video_file):
    is_fight, confidence = predict_violence(video_file)
    result_text = f"🔴 Violence Detected! (Confidence: {confidence:.2f})" if is_fight else f"🟢 No Violence Detected. (Confidence: {confidence:.2f})"
    return result_text

with gr.Blocks() as demo:
    gr.Markdown("## 🎥 Violence Detection in Video")
    gr.Markdown("Upload a video and click 'Detect Violence'. The model will analyze the clip and display the result.")

    video_input = gr.Video(label="Upload a short video", type="filepath")
    output_text = gr.Textbox(label="Prediction Result", interactive=False)
    predict_button = gr.Button("🚨 Detect Violence")

    predict_button.click(fn=gradio_predict, inputs=video_input, outputs=output_text)

demo.launch(debug=False, share=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully unin

RecursionError: maximum recursion depth exceeded while calling a Python object